## Домашняя работа к уроку 6

### Задача look-alike

**Домашнее задание**

1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. Сделать ```feature engineering```
3. Обучить любой классификатор (какой вам нравится)
4. Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. Применить ```random negative sampling``` для построения классификатора в новых условиях
6. Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

Информация о данных:
- Industrial Risk: {P,A,N}
- Management Risk: {P,A,N}
- Financial Flexibility: {P,A,N}
- Credibility: {P,A,N}
- Competitiveness: {P,A,N}
- Operating Risk: {P,A,N}
- Class: {B,NB}

---

In [2]:
# Загрузка необходимых библиотек

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [3]:
# Загрузка данных

df_bankruptcy = pd.read_csv("Qualitative_Bankruptcy.data.txt", header = None)
df_bankruptcy.columns = ['Industrial Risk', 'Management Risk', 'Financial Flexibility', 
              'Credibility','Competitiveness', 'Operating Risk','Class']

# Проверка

df_bankruptcy.head(10)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,Class
0,P,P,A,A,A,P,NB
1,N,N,A,A,A,N,NB
2,A,A,A,A,A,A,NB
3,P,P,P,P,P,P,NB
4,N,N,P,P,P,N,NB
5,A,A,P,P,P,A,NB
6,P,P,A,P,P,P,NB
7,P,P,P,A,A,P,NB
8,P,P,A,P,A,P,NB
9,P,P,A,A,P,P,NB


**Анализ данных:** 6 признаков и 1 целевая - Class

In [9]:
# Формула для проверки качества модели

def evaluate_results(y_test, y_predict):
    
    print('Classification results:')
    
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0))
    
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0))
    
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0))
    
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0))
    
    return [f1,roc,rec,prc]

---

**Feature engineering**

In [5]:
mapping = {'P': 1,'A': 0,'N': -1,'B': -1,'NB': 1}

df_new = pd.DataFrame()

# Mapping

for column in df_bankruptcy.columns:
    df_new[column] = df_bankruptcy[column].map(mapping)
    
# Cуммарный риск и сумма (Credibility + Competitiveness)

df_new['F1'] = (df_new['Industrial Risk'] + df_new['Management Risk'] + df_new['Operating Risk'])
df_new['F2'] = (df_new['Credibility'] + df_new['Competitiveness'])

# Целевой столбец

columns = list(df_new.columns)
a, b = columns.index('Class'), columns.index('F2')
columns[b], columns[a] = columns[a], columns[b]
df_new = df_new[columns]

# Проверка

df_new.head(10)

,Industrial Risk,Management Risk,Financial Flexibility,Credibility,Competitiveness,Operating Risk,F2,F1,Class
0,1,1,0,0,0,1,0,3,1
1,-1,-1,0,0,0,-1,0,-3,1
2,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,2,3,1
4,-1,-1,1,1,1,-1,2,-3,1
5,0,0,1,1,1,0,2,0,1
6,1,1,0,1,1,1,2,3,1
7,1,1,1,0,0,1,0,3,1
8,1,1,0,1,0,1,1,3,1
9,1,1,0,0,1,1,1,3,1


In [6]:
# Проверка целевой

df_new['Class'].value_counts()

 1    143
-1    107
Name: Class, dtype: int64

---

**Обучение классификатора - KNeighbors**

In [7]:
# Сплит данных на целевую и остальное

y = df_new['Class']
X = df_new.drop(columns = 'Class')

# Сплит на train и test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [8]:
# Обучение

model = KNeighborsClassifier(n_neighbors = 2)

model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [10]:
# Проверка качества

result = evaluate_results(y_test, y_predict)
result.append('step4')

A = pd.DataFrame([result],columns = ['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 100.00%
roc: 100.00%
recall: 100.00%
precision: 100.00%


---

**Разделить ваш набор данных на два множества: P (positives) и U (unlabeled)**

Брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [11]:
share = .05

mod_data = df_new.copy()

# Get the indices of the positives samples
P = np.where(mod_data['Class'].values == 1)[0]

# Shuffle them
np.random.shuffle(P)

# Leave just share% of the positives marked
P_len = int(np.ceil(share * len(P)))
print(f'Using {P_len}/{len(P)} as positives and unlabeling the rest')
P_sample = P[:P_len]

Using 8/143 as positives and unlabeling the rest


In [12]:
# Столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

mod_data['class_test'] = -1
mod_data.loc[P_sample,'class_test'] = 1

print('target variable:\n', mod_data['class_test'].value_counts())

target variable:
 -1    242
 1      8
Name: class_test, dtype: int64


In [13]:
X = mod_data.iloc[:,:-2].values

y_labeled = mod_data.class_test.values
y_positive = mod_data.Class.values

---

**Применить random negative sampling для построения классификатора в новых условиях**

In [14]:
mod_data = mod_data.sample(frac = 1)

neg_sample = mod_data[mod_data['class_test'] == -1][:len(mod_data[mod_data['class_test'] == 1])]
sample_test = mod_data[mod_data['class_test'] == -1][len(mod_data[mod_data['class_test'] == 1]):]
pos_sample = mod_data[mod_data['class_test'] == 1]
print(neg_sample.shape, pos_sample.shape)

sample_train = pd.concat([neg_sample, pos_sample]).sample(frac = 1)

(8, 10) (8, 10)


In [15]:
model = KNeighborsClassifier(n_neighbors=2)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

result = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
result.append('last step')
B = pd.DataFrame([result], columns = ['f1', 'roc', 'recall', 'precision','Model'])

Classification results:
f1: 94.02%
roc: 94.21%
recall: 89.39%
precision: 99.16%


---

**Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)**

In [16]:
AB = A.append(B)
AB = AB[['Model','f1', 'roc', 'recall', 'precision']]

# Проверка

AB

,Model,f1,roc,recall,precision
0,step4,1.000000,1.000000,1.000000,1.000000
0,last step,0.940239,0.942068,0.893939,0.991597


Качество решения из пункта 4 выше из-за наибольшей выборки

---

**Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)**

**Результат:** Разница становится заметной при низких значениях выборки (от 5%) и скорее всего обусловлено сильной связью между факторами и предсказанием